### 在Cython中使用C++
作者 --  tushushu  
项目地址 -- https://github.com/tushushu/flying-python

## 1. 在Jupyter Notebook上使用C++ 
- 首先加载Cython扩展，使用魔术命令  ``%load_ext Cython``
- 接下来运行Cython代码，使用魔术命令  ``%%cython --cplus``

In [1]:
%load_ext Cython

In [7]:
%%cython --cplus
from libcpp.vector cimport vector
cdef vector[int] v = [1, 2, 3, 4, 5]
print(v)

[1, 2, 3, 4, 5]


## 2. C++和Python类型的相互转换

## 3. 使用C++ Vector

## 4. 使用C++ Unordered Map